In [1]:
import os
import networkx as nx
import tarfile
import pandas as pd
import shutil
import numpy as np
import yaml
from getpass import getpass

In [2]:
# Define destination for the generated samples
dataset_path = "dataset"
#paths relative to data folder


# ---------------ORIGINAL------------------------
original_directory = 'original'
graphs_path = os.path.join(original_directory,"graphs")
routings_path = os.path.join(original_directory,"routings")
tm_path =  os.path.join(original_directory,"tm_extracted")


# ---------------------PREDICTION-------------------
prediction_directory = 'prediction'
graphs_pred_path = os.path.join(prediction_directory,'graphs')
routings_pred_path = os.path.join(prediction_directory,'routings')
tm_pred_path = os.path.join(prediction_directory,'tm')
pkts_info = os.path.join(prediction_directory,'pkts_info')

# ---------------------SIM-------------------
simulation_directory = 'simulation'
graphs_sim_path = os.path.join(simulation_directory,'graphs')
routings_sim_path = os.path.join(simulation_directory,'routings')
tm_sim_path = os.path.join(simulation_directory,'tm')
simulation_file = os.path.join(simulation_directory, 'simulation.txt')
# Path to simulator file
try:
    os.chdir(f"{os.getcwd()}/{dataset_path}")
except:
    pass

In [3]:
# shutil.rmtree(os.path.join(simulation_directory,'results'))
try:
    shutil.rmtree(os.path.join(simulation_directory,'tmp'))
except:
    pass

In [4]:
os.makedirs(prediction_directory,exist_ok=True)
os.makedirs(pkts_info,exist_ok=True)
os.makedirs(graphs_pred_path,exist_ok=True)
os.makedirs(routings_pred_path,exist_ok=True)
os.makedirs(simulation_directory,exist_ok=True)
os.makedirs(graphs_sim_path,exist_ok=True)
os.makedirs(routings_sim_path,exist_ok=True)
os.makedirs(tm_sim_path,exist_ok=True)

Graph

In [5]:
def write_graph(graphs: list, path: str):
    for graph_filename, graph in graphs.items():
        f = os.path.join(path, graph_filename)
        if not os.path.isfile(f):
            # Creates a new file
            with open(f, 'w') as _:
                pass
        nx.write_gml(graph,f)

def modify_band(graph: nx.DiGraph):
    for edge in graph.edges(data=True):
        params = edge[2]
        params['bandwidth'] = int(params['bandwidth'] / 1000)
    
    return graph

In [6]:
graphs_sim = {}
graphs = {}

buffer = 32_000

for filename in os.listdir(graphs_path):
    f = os.path.join(graphs_path, filename)
    graph = nx.read_gml(f)
    graph = modify_band(graph)
    graphs[filename] = graph
    if filename[0] == 'g':
        sim_graph = graph.to_undirected()
        sim_graph = nx.Graph(sim_graph)
        nx.set_node_attributes(sim_graph, buffer, "bufferSizes")
        graphs_sim[filename] = sim_graph


write_graph(graphs_sim,graphs_sim_path)
write_graph(graphs,graphs_pred_path)


Routing

In [7]:
for filename in os.listdir(routings_path):
    f = os.path.join(routings_path, filename)
    os.popen(f'copy {f} {os.path.join(routings_pred_path,filename)}')
    if filename[0] == 'R':
        new = os.path.join(routings_sim_path, filename)
        lines = []
        with open(f, 'r') as fd:
            for line in fd:
                lines.append(line.replace(' ',','))
        with open(new, 'w') as fd:
            for line in lines:
                fd.write(line)


TM
- Multi Burst (MB) flow:
- Packet size in bytes
- Bandwidth rate in bps during the burst
- Number of packets of each burst
- Number of burst during the experiment (this value can be ignored)
- Inter burst gap in microseconds
- Inter stream gap in microseconds. Time to start the first burst. Always 0 for these datasets.
- ToS. Always 0 for these datasets.

/////////////

Constant Bit Rate (CBR) flow:
- Packet size in bytes
- Bandwidth rate in bps
- ToS. Always 0 for these datasets.

In [8]:
def make_tarfile(output_filepath: os.path, path_list: list):
    with tarfile.open(output_filepath, "w:gz") as tar:
        for path in path_list:
            tar.add(path, arcname=os.path.join(os.path.basename(output_filepath)[:-7],os.path.basename(path)))

def complete_sim_file(sim_file: os.path,filepath: os.path):
    data = pd.read_csv(filepath, sep=";",header=None,dtype="string")
    with open(sim_file, 'a') as file:
        for _, row in data.iterrows():
            g = f'graphs/{row[1]}'
            r = f'routings/{row[2]}'
            t = f'tm/tm-{row[0]}.txt'
            file.write(f'{g},{r},{t}\n')

def convert_line(line: str):
    sign_pos = line.find('|')
    line_start = '10,' + line[:sign_pos + 1]
    res_table = line[sign_pos + 1:].split(';')

    for ii, entry in enumerate(res_table):
        flow_measures = entry.split(',')
        flow_measures[0] = str(float(flow_measures[0]) * 1000)
        for i in range(8):
            flow_measures.append('0')
            res_table[ii] = ','.join(flow_measures)
    return '\n' + line_start + ';'.join(res_table) +';'

def check_sim_results(dataset_name: str, orignal_input_file_lines: list):
    with open(os.path.join(prediction_directory,'experimentResults.txt'), 'w',encoding='utf-8',newline='\n') as file:
        pass
    with open(os.path.join(prediction_directory,'input_files.txt'), 'w',encoding='utf-8',newline='\n') as file:
        pass
    results_dir = os.path.join(simulation_directory,'results',dataset_name)

    initial_line = True
    for filename in os.listdir(results_dir):
        f = os.path.join(results_dir, filename)
        if os.path.isfile(f) and '.tar.gz' in f:
            tar = tarfile.open(f, 'r:gz')
            dir_info = tar.next()
            results_file = tar.extractfile(dir_info.name+"/simulationResults.txt")

            results_lines = []
            while (True):
                results_line = results_file.readline().decode()[:-2]
                if (len(results_line) == 0):
                    break
                results_lines.append(convert_line(results_line))

            if initial_line:
                results_lines[0] = results_lines[0][1:]
                initial_line = False

            with open(os.path.join(prediction_directory,'experimentResults.txt'), 'a',encoding='utf-8',newline='\n') as file:
                for entry in results_lines:
                    file.write(entry)

            input_file = tar.extractfile(dir_info.name+"/input_files.txt")
            results_lines = []
            while (True):
                results_line = input_file.readline().decode()
                if (len(results_line) == 0):
                    break
                pos = results_line.find(';')
                index = int(results_line[0:pos])
                results_lines.append(orignal_input_file_lines[index])
            with open(os.path.join(prediction_directory,'input_files.txt'), 'a',encoding='utf-8',newline='\n') as file:
                for entry in results_lines:
                    file.write(entry)

In [9]:
def yaml_config(dataset_name: str):
    conf_file = os.path.join(simulation_directory,"conf.yml")
    conf_parameters = {
        "threads": 6,# Number of threads to use 
        "dataset_name": dataset_name, # Name of the dataset. It is created in <training_dataset_path>/results/<name>
        "samples_per_file": 10, # Number of samples per compressed file
        "rm_prev_results": "n", # If 'y' is selected and the results folder already exists, the folder is removed.
        "write_pkt_info": "n", # If 'y' is selected, a file per simulation is created in the pkts_info folder of the dataset. This file contain a line per packet with the following data: src_id dst_id flow_id tos timestamp(ns) pkt_size[ delay(ns)]
    }

    with open(conf_file, 'w') as fd:
        yaml.dump(conf_parameters, fd)

def dest_path():
    graphs_path = "graphs"
    routings_path = "routings"
    tm_path = "tm"
    if os.path.isdir(simulation_directory):
        print ("Destination path already exists. Files within the directory may be overwritten.")
    else:
        os.makedirs(os.path.join(simulation_directory,graphs_path))
        os.mkdir(os.path.join(simulation_directory,routings_path))
        os.mkdir(os.path.join(simulation_directory,tm_path))

def docker_cmd_run(training_dataset_path):
    raw_cmd = f"docker run --rm --mount type=bind,src={os.path.join(os.getcwd(),training_dataset_path)},dst=/data bnnupc/bnnetsimulator"
    terminal_cmd = raw_cmd
    if os.name != 'nt': # Unix, requires sudo
        print("Superuser privileges are required to run docker. Introduce sudo password when prompted")
        terminal_cmd = f"echo {getpass()} | sudo -S " + raw_cmd
        raw_cmd = "sudo " + raw_cmd
    print(raw_cmd)
    !{terminal_cmd}
    return raw_cmd, terminal_cmd

def run_sim(dataset_name:str):

    yaml_config(dataset_name)
    # dest_path()
    docker_cmd_run(simulation_directory)

In [10]:
# parameter to reduce badnwith to have feasible simulation time
band_param = 10000

my_cols = ['src','dst','packet_size','avg_band','no_of_packetrs','no_of_bursts','burst gap','stream gap','ToS']

counter = 375
for filename in os.listdir(original_directory):
    f = os.path.join(original_directory, filename)
    if os.path.isfile(f):
        with open(simulation_file,'w'):
            pass
        os.makedirs(tm_pred_path,exist_ok=True)
        tm_dir = os.path.join(original_directory,filename[:-7],filename[:-7],'tm')
        if not os.path.isdir(tm_dir):
            file = tarfile.open(f)
            file.extractall(f[:-7])
        for tm_file_name in os.listdir(tm_dir):
            tm_file = os.path.join(tm_dir, tm_file_name)
            data = pd.read_csv(tm_file, sep=",|;",header=None,names=my_cols,dtype="string",engine='python')
            max_load = data['src'][0]
            data['avg_band'] = data['avg_band'].astype('Float64')
            data = data[1:]
            data['avg_band'] = data['avg_band'] / band_param

            tm_data = data.copy()
            tm_data.insert(0,"src_dst_size",tm_data.src.str.cat([tm_data.dst,tm_data.packet_size],sep=';'))
            tm_data = tm_data.drop(columns=['src','dst','packet_size'])
            pred_file = os.path.join(tm_pred_path, f'tm-{counter}.txt')
            tm_data.to_csv(pred_file,header=False,index=False,encoding='utf-8')

            with open(pred_file, 'r',encoding='utf-8',newline='\n') as file: 
                lines = file.readlines() 

            with open(pred_file, 'w',encoding='utf-8',newline='\n') as file:
                file.write(max_load + '\n')
                for line in lines:
                    file.write("\n".join(line.splitlines())+ '\n')

            new_df = pd.DataFrame(columns=['src','dst','avg_band','packet_size','ToS'])
            new_df['src'] = data.src
            new_df['dst'] = data.dst
            new_df['avg_band'] = data['avg_band']
            new_df['packet_size'] = data['packet_size']
            new_df['ToS'] = data['ToS']
            new_df.insert(3,"time_distribution",'0')
            new_df.insert(4,"pkt_dist",'0')
            new_df.insert(6,"prob_n",'1')

            new_df.avg_band = np.ceil(new_df.avg_band).astype(int)
            sim_file = os.path.join(tm_sim_path, f'tm-{counter}.txt')
            new_df.to_csv(sim_file,header=False,index=False)

            counter += 1

        tar_file = os.path.join(f[:-7],filename[:-7])
        orig_input_file = os.path.join(tar_file,'input_files.txt')

        orignal_input_file_lines = []
        with open(orig_input_file, 'r',encoding='utf-8',newline='\n') as file: 
            orignal_input_file_lines = file.readlines() 

        complete_sim_file(simulation_file,orig_input_file)

        
        # ///////////
        
        run_sim(filename[:-7])


        check_sim_results(filename[:-7],orignal_input_file_lines)
        exp_results = os.path.join(prediction_directory,'experimentResults.txt')
        pred_input = os.path.join(prediction_directory,'input_files.txt')
        path_list = [tm_pred_path,exp_results,pred_input]
        rar_name = os.path.join(prediction_directory,filename)
        make_tarfile(rar_name,path_list)
        shutil.rmtree(tm_pred_path)
        os.remove(exp_results)
        os.remove(pred_input)
        shutil.rmtree(f[:-7])
            

docker run --rm --mount type=bind,src=d:\GNN\dataset\simulation,dst=/data bnnupc/bnnetsimulator


INFO:root:4: OK
INFO:root:0: OK
INFO:root:1: OK
INFO:root:5: OK
INFO:root:2: OK
INFO:root:6: OK
INFO:root:8: OK
INFO:root:3: OK
INFO:root:9: OK
INFO:root:10: OK
INFO:root:7: OK
INFO:root:13: OK
INFO:root:11: OK
INFO:root:14: OK
INFO:root:16: OK
INFO:root:15: OK
INFO:root:12: OK
INFO:root:21: OK
INFO:root:17: OK
INFO:root:19: OK
INFO:root:22: OK
INFO:root:18: OK
INFO:root:20: OK
INFO:root:24: OK
INFO:root:23: OK


docker run --rm --mount type=bind,src=d:\GNN\dataset\simulation,dst=/data bnnupc/bnnetsimulator


INFO:root:1: OK
INFO:root:4: OK
INFO:root:6: OK
INFO:root:2: OK
INFO:root:0: OK
INFO:root:3: OK
INFO:root:5: OK
INFO:root:7: OK
INFO:root:9: OK
INFO:root:8: OK
INFO:root:11: OK
INFO:root:12: OK
INFO:root:13: OK
INFO:root:15: OK
INFO:root:14: OK
INFO:root:10: OK
INFO:root:19: OK
INFO:root:20: OK
INFO:root:16: OK
INFO:root:17: OK
INFO:root:21: OK
INFO:root:18: OK
INFO:root:22: OK
INFO:root:24: OK
INFO:root:23: OK


docker run --rm --mount type=bind,src=d:\GNN\dataset\simulation,dst=/data bnnupc/bnnetsimulator


...

INFO:root:3: OK
INFO:root:4: OK
INFO:root:5: OK
INFO:root:2: OK
INFO:root:0: OK
INFO:root:1: OK
INFO:root:6: OK
INFO:root:8: OK
INFO:root:9: OK
INFO:root:13: OK
INFO:root:7: OK
INFO:root:11: OK
INFO:root:12: OK
INFO:root:10: OK
INFO:root:14: OK
INFO:root:15: OK
INFO:root:16: OK
INFO:root:18: OK
INFO:root:20: OK
INFO:root:19: OK
INFO:root:22: OK
INFO:root:17: OK
INFO:root:23: OK
INFO:root:24: OK
INFO:root:21: OK


docker run --rm --mount type=bind,src=d:\GNN\dataset\simulation,dst=/data bnnupc/bnnetsimulator


ERROR:root:2: Error using traffic matrix file /data/tm/tm-652.txt at line 17. Average bandwidth of the path should be bigger than 10: 8.0 
INFO:root:4: OK
INFO:root:0: OK
INFO:root:6: OK
INFO:root:1: OK
INFO:root:5: OK
INFO:root:8: OK
INFO:root:9: OK
INFO:root:3: OK
INFO:root:11: OK
INFO:root:7: OK
INFO:root:10: OK
INFO:root:14: OK
INFO:root:12: OK
INFO:root:15: OK
INFO:root:20: OK
INFO:root:19: OK
INFO:root:13: OK
INFO:root:17: OK
INFO:root:22: OK
INFO:root:23: OK
INFO:root:18: OK
INFO:root:24: OK
INFO:root:16: OK
INFO:root:21: OK


docker run --rm --mount type=bind,src=d:\GNN\dataset\simulation,dst=/data bnnupc/bnnetsimulator


INFO:root:5: OK
INFO:root:0: OK
INFO:root:4: OK
INFO:root:2: OK
INFO:root:3: OK
INFO:root:6: OK
INFO:root:1: OK
INFO:root:9: OK
INFO:root:7: OK
INFO:root:8: OK
INFO:root:10: OK
INFO:root:15: OK
INFO:root:12: OK
INFO:root:11: OK
INFO:root:14: OK
INFO:root:13: OK
INFO:root:19: OK
INFO:root:20: OK
INFO:root:16: OK
INFO:root:18: OK
INFO:root:17: OK
INFO:root:23: OK
INFO:root:21: OK
INFO:root:24: OK
INFO:root:22: OK
